# Importation

In [1]:
import geopandas as gpd
from osgeo import gdal
import os
import numpy as np
import sys
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw

# Raster 60 bandes

In [2]:
#  Définition des paramètres 

input_raster_dir = "/home/onyxia/work/data/images"
L_images = sorted(os.listdir(input_raster_dir))
L_images.remove('.keep')
ref_raster_path = "/home/onyxia/work/data/images/SENTINEL2B_20220125-105852-948_L2A_T31TCJ_C_V3-0_SRE_B2.tif"
shapefile_path = "/home/onyxia/work/data/project/emprise_etude.shp"
mask_foret_path = "/home/onyxia/Projet_Teledec/results/data/img_pretraitees/masque_foret.tif"
output_dir = "/home/onyxia/work/output"

os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Construction array

x,y = rw.get_image_dimension(rw.open_image(ref_raster_path))[:2]
bandes = 60
array_tot = np.zeros((x,y,bandes))

L_array = []
for i,img in enumerate(L_images):
    path = os.path.join(input_raster_dir,img) 
    array = rw.load_img_as_array(path)
    L_array.append(array)

In [3]:
gdal.SetConfigOption("GDAL_CACHEMAX", "1000")  # Cache en Mo

In [4]:
import numpy as np
from osgeo import gdal

def resample_to_resolution(src_array, src_transform, src_projection, target_resolution, target_shape):
    """
    Rééchantillonne un tableau numpy à une nouvelle résolution en utilisant GDAL.
    """
    mem_driver = gdal.GetDriverByName("MEM")
    target_rows, target_cols = target_shape

    # Créer une image GDAL en mémoire pour l'entrée
    src_ds = mem_driver.Create("", src_array.shape[1], src_array.shape[0], 1, gdal.GDT_Float32)
    src_ds.GetRasterBand(1).WriteArray(src_array)
    src_ds.SetGeoTransform(src_transform)
    src_ds.SetProjection(src_projection)

    # Créer une image GDAL en mémoire pour la sortie
    target_ds = mem_driver.Create("", target_cols, target_rows, 1, gdal.GDT_Byte)
    target_transform = (
        src_transform[0], target_resolution, 0,
        src_transform[3], 0, -target_resolution
    )
    target_ds.SetGeoTransform(target_transform)
    target_ds.SetProjection(src_projection)

    # Créer les options de rééchantillonnage
    warp_options = gdal.WarpOptions(
        resampleAlg="bilinear",  # Algorithme bilinéaire
        multithread=True         # Utilisation de plusieurs threads pour accélérer
    )

    # Rééchantillonner avec GDAL Warp
    gdal.Warp(target_ds, src_ds, options=warp_options)
    
    # Lire et convertir en UInt8
    resampled_array = target_ds.GetRasterBand(1).ReadAsArray().astype(np.uint8)
    return resampled_array

def process_image(image_path, target_resolution, target_shape):
    """
    Charge et rééchantillonne une image entière.
    """
    ds = gdal.Open(image_path)
    array = ds.GetRasterBand(1).ReadAsArray()
    transform = ds.GetGeoTransform()
    projection = ds.GetProjection()

    resampled_array = resample_to_resolution(array, transform, projection, target_resolution, target_shape)
    return resampled_array

def concatenate_images(image_paths, target_resolution):
    """
    Concatène toutes les images après rééchantillonnage.
    """
    target_shape = None
    resampled_arrays = []

    for i, path in enumerate(image_paths):
        # Charger la première image pour calculer la forme cible
        if target_shape is None:
            ds = gdal.Open(path)
            transform = ds.GetGeoTransform()
            
            # Calculer les dimensions cibles
            cols = int((ds.RasterXSize * transform[1]) / target_resolution)
            rows = int((ds.RasterYSize * abs(transform[5])) / target_resolution)
            target_shape = (rows, cols)

        # Rééchantillonner chaque image
        resampled_array = process_image(path, target_resolution, target_shape)
        resampled_arrays.append(resampled_array)

        # Suivi de la progression
        print(f"Image {i+1}/{len(image_paths)} traitée")

    # Concaténer les tableaux après rééchantillonnage
    concatenated_array = np.stack(resampled_arrays, axis=0)
    return concatenated_array


# # Exemple d'utilisation
# image_paths = ["image1.tif", "image2.tif", "image3.tif"]  # Chemins vers les images Sentinel-2
# target_resolution = 10  # Résolution cible en mètres
# result = concatenate_images(image_paths, target_resolution)

In [5]:
image_paths = []
for i in L_images:
    image_paths.append(os.path.join('/home/onyxia/work/data/images',i))
target_resolution = 10  # Résolution cible en mètres
output_file = os.path.join(output_dir,"Serie_temp_S2_allbands.tif")  # Nom de fichier de sortie

In [ ]:
dataset_exist = rw.open_image(image_paths[2])
rw.get_image_dimension(dataset_exist)

In [ ]:
#Def d'un dataset vierge pour l'écriture
driver = gdal.GetDriverByName('GTiff')

out_filename = 'dataset_vierge.tiff'
nb_col,nb_ligne,nb_band = rw.get_image_dimension(dataset_exist)
gdal_dtype = gdal.GDT_Byte
output_data_set = driver.Create(out_filename, nb_col, nb_ligne, nb_band, gdal_dtype)

#Ouverture de ce dataset
dataset = rw.open_image('/home/onyxia/work/Projet_Teledec/scripts/dataset_vierge.tiff')

In [6]:
result = concatenate_images(image_paths, target_resolution)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Image 1/60 traitée
Image 2/60 traitée
Image 3/60 traitée
Image 4/60 traitée
Image 5/60 traitée
Image 6/60 traitée
Image 7/60 traitée
Image 8/60 traitée
Image 9/60 traitée
Image 10/60 traitée
Image 11/60 traitée
Image 12/60 traitée
Image 13/60 traitée
Image 14/60 traitée
Image 15/60 traitée
Image 16/60 traitée
Image 17/60 traitée
Image 18/60 traitée
Image 19/60 traitée
Image 20/60 traitée
Image 21/60 traitée
Image 22/60 traitée
Image 23/60 traitée
Image 24/60 traitée
Image 25/60 traitée
Image 26/60 traitée
Image 27/60 traitée
Image 28/60 traitée
Image 29/60 traitée
Image 30/60 traitée
Image 31/60 traitée
Image 32/60 traitée
Image 33/60 traitée
Image 34/60 traitée
Image 35/60 traitée
Image 36/60 traitée
Image 37/60 traitée
Image 38/60 traitée
Image 39/60 traitée
Image 40/60 traitée
Image 41/60 traitée
Image 42/60 traitée
Image 43/60 traitée
Image 44/60 traitée
Image 45/60 traitée
Image 46/60 traitée
Image 47/60 traitée
Image 48/60 traitée
Image 49/60 traitée
Image 50/60 traitée
Image 51/

In [ ]:
dataset = rw.open_image(ref_raster_path)
rw.write_image(out_filename = "Serie_temp_S2_allbands.tif", array = result, gdal_dtype = gdal.GDT_Byte, data_set = dataset)

# Clip Raster

In [ ]:
#Façon potentiel pour clip un raster grâce à l'emprise
#Pour l'instant ne pas faire tourner

# Charger le vecteur avec Geopandas
gdf = gpd.read_file("/home/onyxia/work/data/project/emprise_etude.shp").to_crs("EPSG:2154")

# Extraire le GeoJSON sous forme de string
geojson_str = gdf.to_json()

# Appliquer le clip avec GDAL
resolution = 10  # Résolution (10 m)
gdal.Warp(
    "/home/onyxia/work/test.tif",     #Chemin fichier en sortie
    "/home/onyxia/work/data/images/SENTINEL2B_20220125-105852-948_L2A_T31TCJ_C_V3-0_SRE_B2.tif", #Chemin fichier en entrée
    format="GTiff",
    cutlineDSName=geojson_str,  # Passer directement le GeoJSON
    cropToCutline=True,
    dstSRS="EPSG:2154",
    xRes=resolution,
    yRes=resolution,
    dstNodata=0
)